In [2]:
#encoding=utf-8
import pandas as pd
import numpy as np

In [3]:
# path = '/home/kai/data/jiangning/Chatbot_1/Chatbot1.0/data/'
path = '../../data/others/'
data = pd.read_csv(path + 'irrelevant_response.csv', encoding='utf8')
data = data.rename(index=str, columns={'文本':'text','类别':'label','对策':'strategy'}).dropna()
model_list = ['Idclassfier','ifknowdebtor','causedebt','willingtopay','cutdebt','installation','setdueday']
print(data.shape)
data.head(30)

(2750, 10)


,text,label,Idclassfier,ifknowdebtor,causedebt,willingtopay,cutdebt,installation,setdueday,strategy
0,只愿还本金,讨价还价,1.0,1.0,0.0,0.0,0.0,0.0,0.0,willingtopay
1,只愿还本金,讨价还价,1.0,1.0,0.0,0.0,0.0,0.0,0.0,willingtopay
2,只还本金,讨价还价,1.0,1.0,0.0,0.0,0.0,0.0,0.0,willingtopay
3,之前没钱,讨价还价,1.0,1.0,0.0,0.0,0.0,0.0,0.0,willingtopay
4,正在银行,讨价还价,1.0,1.0,0.0,0.0,0.0,0.0,0.0,willingtopay
5,正在想办法,讨价还价,1.0,1.0,0.0,0.0,0.0,0.0,0.0,willingtopay
6,正在凑钱,讨价还价,1.0,1.0,0.0,0.0,0.0,0.0,0.0,willingtopay
7,这两天,讨价还价,1.0,1.0,0.0,0.0,0.0,0.0,0.0,willingtopay
8,这卡是我刚办理的,讨价还价,1.0,1.0,0.0,0.0,0.0,0.0,0.0,willingtopay
9,这个星期我就处理,讨价还价,1.0,1.0,0.0,0.0,0.0,0.0,0.0,willingtopay


In [32]:
for each_model in model_list:
    each_data = data[data[each_model]==0].copy()

In [5]:
# data[data['label']=='不愿还款']
data[data['label']=='不愿配合']
# data[data['label']=='帮还钱']
# data[data['label']=='讨价还价']

,text,label,Idclassfier,ifknowdebtor,causedebt,willingtopay,cutdebt,installation,setdueday,strategy
2070,自己找他,不愿配合,0.0,0.0,0.0,0.0,0.0,0.0,0.0,强硬施压
2071,自己找他,不愿配合,0.0,0.0,0.0,0.0,0.0,0.0,0.0,强硬施压
2072,自己找,不愿配合,0.0,0.0,0.0,0.0,0.0,0.0,0.0,强硬施压
2073,自己想,不愿配合,0.0,0.0,0.0,0.0,0.0,0.0,0.0,强硬施压
2074,自己去营业厅查,不愿配合,0.0,0.0,0.0,0.0,0.0,0.0,0.0,强硬施压
2075,自己去查,不愿配合,0.0,0.0,0.0,0.0,0.0,0.0,0.0,强硬施压
2076,转告不了,不愿配合,0.0,0.0,0.0,0.0,0.0,0.0,0.0,强硬施压
2077,直接挂断,不愿配合,0.0,0.0,0.0,0.0,0.0,0.0,0.0,强硬施压
2078,直接挂！,不愿配合,0.0,0.0,0.0,0.0,0.0,0.0,0.0,强硬施压
2079,直接挂,不愿配合,0.0,0.0,0.0,0.0,0.0,0.0,0.0,强硬施压


In [16]:
(data['label'].unique())

array(['讨价还价', '确认数额', '请求重复', '请求等下打来', '模糊确认', '回问身份', '还款方式', '故意岔开话题',
       '承认是自己身份', '承认是别人身份', '不愿配合', '不愿还款', '不承认欠款', '帮还钱'], dtype=object)

In [21]:
data.drop('text',axis=1).drop_duplicates(['label','Idclassfier','ifknowdebtor','causedebt','willingtopay','cutdebt','installation','setdueday','strategy'])

,label,Idclassfier,ifknowdebtor,causedebt,willingtopay,cutdebt,installation,setdueday,strategy
0,讨价还价,1.0,1.0,0.0,0.0,0.0,0.0,0.0,willingtopay
362,确认数额,0.0,0.0,0.0,0.0,0.0,0.0,0.0,欠款信息在app可查，重复问题
399,确认数额,1.0,1.0,0.0,0.0,0.0,0.0,0.0,欠款信息在app可查，重复问题
408,请求重复,0.0,0.0,0.0,0.0,0.0,0.0,0.0,重复问题
798,请求等下打来,1.0,1.0,0.0,0.0,0.0,0.0,0.0,之后重新打
804,请求等下打来,0.0,0.0,0.0,0.0,0.0,0.0,0.0,之后重新打
1404,模糊确认,1.0,1.0,0.0,0.0,0.0,0.0,0.0,强硬施压
1425,回问身份,0.0,0.0,1.0,1.0,1.0,1.0,1.0,身份回答并重复问题
1433,回问身份,0.0,0.0,0.0,1.0,1.0,1.0,1.0,身份回答并重复问题
1539,还款方式,1.0,1.0,0.0,0.0,0.0,0.0,0.0,告知还款方式


In [41]:
# data[data['Idclassfier']==0]

In [26]:
data.label.value_counts()

不愿还款       587
请求等下打来     586
请求重复       389
讨价还价       362
故意岔开话题     311
承认是别人身份    191
回问身份       114
不愿配合        98
确认数额        46
模糊确认        21
承认是自己身份     19
不承认欠款       17
还款方式         6
帮还钱          3
Name: label, dtype: int64